In [52]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import truncnorm
import time
import csv


def get_s_cond_t_params(player_1_sigma, player_2_sigma, player_1_mean, player_2_mean, t_i):
    beta = 3  # från uppgiften

    # Covariance calculations
    covariance_t_given_s = beta
    covariance_ss = np.array([[player_1_sigma, 0], [0, player_2_sigma]])
    A = np.array([1, -1]).reshape([1, 2])
    inv_covariance_ss = np.linalg.inv(covariance_ss)
    ACA = np.matmul(A.T, A) * (1 / covariance_t_given_s)
    covariance_s_cond_t = np.linalg.inv(inv_covariance_ss + ACA)

    # Mean calculations
    player_means = np.array([player_1_mean, player_2_mean]).reshape([2, 1])
    a = covariance_s_cond_t
    b = np.matmul(inv_covariance_ss, player_means)
    c = A.T * (1 / covariance_t_given_s) * t_i
    mean_s_cond_t = np.matmul(a, b + c)

    return mean_s_cond_t, covariance_s_cond_t


def P_s_cond_t(t_i, player_1_mean, player_1_sigma, player_2_mean, player_2_sigma):
    # player_1_mean = 25; player_2_mean = 25; player_1_sigma = 8.3**2; player_2_sigma = 8.3**2

    mean_s_cond_t, cov_s_cond_t = get_s_cond_t_params(player_1_mean=player_1_mean, player_1_sigma=player_1_sigma,
                                                      player_2_mean=player_2_mean, player_2_sigma=player_2_sigma,
                                                      t_i=t_i)

    return np.random.multivariate_normal(mean=mean_s_cond_t.reshape(2), cov=cov_s_cond_t,
                                         check_valid='warn', tol=1e-8)


def P_t_cond_s(s_i, t_game):
    s_diff = s_i[0] - s_i[1]
    beta = 3
    t_sigma = beta

    if t_game > 0:  # case for when y=1
        a, b = (0 - s_diff) / t_sigma, np.inf
        t = truncnorm.rvs(a, b) * t_sigma + s_diff
        return t
    elif t_game < 0:  # case for when y=-1
        a, b = -np.inf, (0 - s_diff) / t_sigma
        t = truncnorm.rvs(a, b) * t_sigma + s_diff
        return t
    else:
        print("ERROR, TIES PRESENTLY NOT ALLOWED")


        
def gibbs_sampler(L, player_1_stats, player_2_stats, t_game):
    player_1_mean, player_1_sigma = player_1_stats
    player_2_mean, player_2_sigma = player_2_stats
    
    s_i = [player_1_mean, player_2_mean]

    t_obs = np.zeros(L)
    s_obs = np.zeros([L, 2])

    for i in range(L):
        t_i_plus_1 = P_t_cond_s(s_i, t_game=t_game)
        s_i_plus_1 = P_s_cond_t(t_i_plus_1, player_1_mean, player_1_sigma, player_2_mean, player_2_sigma)

        t_obs[i] = t_i_plus_1
        s_obs[i, :] = s_i_plus_1

        s_i = s_i_plus_1
        # plt.scatter(s_obs[:, 0], s_obs[:, 1])
        # plt.pause(0.1)
    # plt.show()

    # plt.plot(s_obs[:, 0]); plt.plot(s_obs[:, 1]); plt.show()
    return s_obs, t_obs



def player_stats_estimate_from_obs(s_obs):
    player_1_stats_estimate = [np.mean(s_obs[:, 0]), np.var(s_obs[:, 0])] # [mean, variance] of samples
    player_2_stats_estimate = [np.mean(s_obs[:, 1]), np.var(s_obs[:, 1])] # [mean, variance] of samples
    
    return player_1_stats_estimate, player_2_stats_estimate


    

def make_stats_dictionary(filename, stats_dictionary, printable):
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile)
        mean = 100; variance = (100 / 3) ** 2  # TrueSkill prior parameters before any games
        gamelist = []
        for row in reader:
            # Create dictionary (=map) with keyword 'team' and value [mean, variance]
            stats_dictionary[row['team1']] = [mean, variance]
            # Add teams and result to list
            gamelist.append([row['team1'], row['team2'], int(row['score1']) - int(row['score2'])])

            if printable == 1:  # Print the whole list
                print(f"{row['team1']} vs {row['team2']}: {row['score1']}-{row['score2']}")  # Access by column header
                      
    #stats_dictionary = random.shuffle(stats_dictionary, random)

    return stats_dictionary, gamelist



# Ranking by mean (should perhaps be improved to mean - 3 * sigma)
def ranking(stats_dictionary):
    sorted_teams = sorted(stats_dictionary.items(), key=lambda x: x[1], reverse=True)
    print("\nList of teams ranked by mean skill in descending order:\n")
    for i in sorted_teams:
        print(i[0], i[1])

def played_before(team1, team2, result_list,tmax):
    for t in range(tmax):
        if result_list[t][0] == team2 and result_list[t][1] == team1:
            return -np.sign(result_list[t][2])
    return -999
                      
def get_form(team, form_dictionary):
    if len(form_dictionary[team]) == 0:
        return 0
    elif len(form_dictionary[team]) < 5:
        return sum(form_dictionary[team]) / len(form_dictionary[team])
    else:
        return sum(form_dictionary[team][5:]) / 5
                      
                      
def predict_winner(team1_string, team2_string, stats_dictionary, form_dictionary, result_list, tmax):
    alfa = 0.05  #team 1 home advantage 5 % boost
                      
    form1 = get_form(team1_string,form_dictionary)
    form2 = get_form(team2_string,form_dictionary)
    beta = 0.1
    
    pb = played_before(team1_string, team2_string, result_list, tmax)
    if  pb == -999:
        gamma = 0
    else: 
        gamma = 0.05
                      
    s1 = stats_dictionary[team1_string][0]
    s2 = stats_dictionary[team2_string][0]

    p1 = s1 + alfa*s1 + beta*form1*s1 + gamma*pb*s1  #prediction score team 1
    p2 = s2 + beta*form2*s2 + gamma*(-pb)*s2   #prediction score team 2
                      
    #diff = stats_dictionary[team1_string][0] - stats_dictionary[team2_string][0] 
    diff = p1 - p2
    print('\t diff: ', diff)
    
    if diff > 0:
        return 1
    elif diff < 0:
        return -1
    else:
        return 1

In [53]:
x = [1,2,3,4,5,6]
x[3:]

[4, 5, 6]

In [54]:
# Make dictionary of team stats (mean and variance) & list of all games with result
stats_dictionary, result_list = make_stats_dictionary('SerieA.csv', stats_dictionary={}, printable=0)
stats_dictionary

{'Chievo': [100, 1111.1111111111113],
 'Lazio': [100, 1111.1111111111113],
 'Torino': [100, 1111.1111111111113],
 'Sassuolo': [100, 1111.1111111111113],
 'Parma': [100, 1111.1111111111113],
 'Empoli': [100, 1111.1111111111113],
 'Bologna': [100, 1111.1111111111113],
 'Atalanta': [100, 1111.1111111111113],
 'Juventus': [100, 1111.1111111111113],
 'Napoli': [100, 1111.1111111111113],
 'Spal': [100, 1111.1111111111113],
 'Udinese': [100, 1111.1111111111113],
 'Inter': [100, 1111.1111111111113],
 'Genoa': [100, 1111.1111111111113],
 'Frosinone': [100, 1111.1111111111113],
 'Fiorentina': [100, 1111.1111111111113],
 'Cagliari': [100, 1111.1111111111113],
 'Roma': [100, 1111.1111111111113],
 'Milan': [100, 1111.1111111111113],
 'Sampdoria': [100, 1111.1111111111113]}

In [55]:
played_before('Juventus','Roma',result_list,10)

-999

In [56]:
np.sign(0)

0

In [57]:
stats_dictionary, result_list = make_stats_dictionary('SerieA.csv', stats_dictionary={}, printable=0)
form_dictionary = dict()
for keys in stats_dictionary:
    form_dictionary[keys] = []
form_dictionary

{'Chievo': [],
 'Lazio': [],
 'Torino': [],
 'Sassuolo': [],
 'Parma': [],
 'Empoli': [],
 'Bologna': [],
 'Atalanta': [],
 'Juventus': [],
 'Napoli': [],
 'Spal': [],
 'Udinese': [],
 'Inter': [],
 'Genoa': [],
 'Frosinone': [],
 'Fiorentina': [],
 'Cagliari': [],
 'Roma': [],
 'Milan': [],
 'Sampdoria': []}

In [58]:
correct_predictions = 0
nr_of_draws = 0



for i in range(len(result_list)):
    # result_list holds [team1, team2, result = score1 - score2]
    print(f"\nResult game {i}:   {result_list[i]}")
    team1 = result_list[i][0]
    team2 = result_list[i][1]
    
    print('team1: ', team1)
    print('team2: ', team2)
    result = result_list[i][2]
    
    # stats_dictionary with keyword 'teamname' and value [mean, variance]
    print(f"Stats before game: {stats_dictionary[team1]}, {stats_dictionary[team2]}")

    if result == 0:  # ignore tied games for now
        print("Game ignored due to tie")
        print(f"Stats after game:  {stats_dictionary[team1]}, {stats_dictionary[team2]}")
        nr_of_draws += 1
        print("Draws: ", nr_of_draws)
        
        form_dictionary[team1].append(0)
        form_dictionary[team2].append(0)
    else:
    
        # prediktera mat resultat och spara prediktion
        pred_result = predict_winner(team1, team2, stats_dictionary, form_dictionary, result_list, tmax=i)
        
        if np.sign(result) == np.sign(pred_result):
                correct_predictions += 1
                print("Correct pred: ", correct_predictions)
        
        print('true result: ', result)
        print('pred result: ', pred_result)
        
        form_dictionary[team1].append(np.sign(result))
        form_dictionary[team2].append(-np.sign(result))
        print('form dic 1:', form_dictionary[team1])
        print('form dic 2:', form_dictionary[team2])
            
        s_obs, t_obs =  gibbs_sampler(L=500,
                                      player_1_stats=stats_dictionary[team1], 
                                      player_2_stats=stats_dictionary[team2],
                                      t_game=result)
        
        burn_in_value = 100
        s_obs = s_obs[burn_in_value:]
        player_1_stats_posterior, player_2_stats_posterior = player_stats_estimate_from_obs(s_obs)
        
        
        # Update team stats so posterior makes new prior
        stats_dictionary[team1] = player_1_stats_posterior
        stats_dictionary[team2] = player_2_stats_posterior


        print(f"Stats AFTER game: {stats_dictionary[team1]}, {stats_dictionary[team2]}")

print()
print("correct_predictions ", correct_predictions)
print("reslist: ", len(result_list))
print("draws: ", nr_of_draws)
print('performance: ', correct_predictions / (len(result_list)-nr_of_draws))



Result game 0:   ['Chievo', 'Juventus', -1]
team1:  Chievo
team2:  Juventus
Stats before game: [100, 1111.1111111111113], [100, 1111.1111111111113]
	 diff:  5.0
true result:  -1
pred result:  1
form dic 1: [-1]
form dic 2: [1]
Stats AFTER game: [88.62213549322823, 638.1325081472946], [110.58383174798092, 641.1851485083411]

Result game 1:   ['Lazio', 'Napoli', -1]
team1:  Lazio
team2:  Napoli
Stats before game: [100, 1111.1111111111113], [100, 1111.1111111111113]
	 diff:  5.0
true result:  -1
pred result:  1
form dic 1: [-1]
form dic 2: [1]
Stats AFTER game: [84.6380309558861, 743.069908112684], [116.7789499984511, 684.548378291495]

Result game 2:   ['Torino', 'Roma', -1]
team1:  Torino
team2:  Roma
Stats before game: [100, 1111.1111111111113], [100, 1111.1111111111113]
	 diff:  5.0
true result:  -1
pred result:  1
form dic 1: [-1]
form dic 2: [1]
Stats AFTER game: [90.32793465489904, 585.50394423304], [109.90643241152827, 540.0106000569027]

Result game 3:   ['Sassuolo', 'Inter', 1]

Stats AFTER game: [127.25403408527133, 374.37116998587595], [107.29358618377326, 341.55878502554157]

Result game 25:   ['Lazio', 'Frosinone', 1]
team1:  Lazio
team2:  Frosinone
Stats before game: [91.0687050235458, 337.88262262821627], [83.59671222239562, 660.1023078567782]
	 diff:  7.098393161092673
Correct pred:  11
true result:  1
pred result:  1
form dic 1: [-1, -1, 1]
form dic 2: [-1, 0, -1]
Stats AFTER game: [102.79608865627924, 249.72936496197136], [60.589255976623335, 273.43570220555415]

Result game 26:   ['Chievo', 'Empoli', 0]
team1:  Chievo
team2:  Empoli
Stats before game: [86.16711191867722, 444.5424741245325], [87.8004937387718, 458.74717863899895]
Game ignored due to tie
Stats after game:  [86.16711191867722, 444.5424741245325], [87.8004937387718, 458.74717863899895]
Draws:  6

Result game 27:   ['Atalanta', 'Cagliari', -1]
team1:  Atalanta
team2:  Cagliari
Stats before game: [117.97866605628914, 575.4068957352068], [87.93930042849213, 564.4378267552016]
	 diff:  46.23

Stats AFTER game: [62.01964484832392, 185.61616991037153], [47.5163426565318, 208.04195006317693]

Result game 47:   ['Milan', 'Atalanta', 0]
team1:  Milan
team2:  Atalanta
Stats before game: [97.94761485287559, 385.1824957703337], [82.47320456687878, 283.181155023421]
Game ignored due to tie
Stats after game:  [97.94761485287559, 385.1824957703337], [82.47320456687878, 283.181155023421]
Draws:  11

Result game 48:   ['Frosinone', 'Juventus', -2]
team1:  Frosinone
team2:  Juventus
Stats before game: [49.25879850421432, 162.44637724163996], [143.7653482691386, 245.64606958752898]
	 diff:  -110.11455455444349
Correct pred:  17
true result:  -2
pred result:  -1
form dic 1: [-1, 0, -1, -1, -1]
form dic 2: [1, 1, 1, 1, 1]
Stats AFTER game: [53.49825575924206, 195.67974822605981], [136.26276771023626, 366.4290966053096]

Result game 49:   ['Inter', 'Fiorentina', 1]
team1:  Inter
team2:  Fiorentina
Stats before game: [116.7219363160888, 122.93384482277084], [126.8523674020177, 128.66534346097

Stats AFTER game: [117.46485992892646, 121.89037215360993], [126.29931974105293, 149.55708788312873]

Result game 68:   ['Sampdoria', 'Spal', 1]
team1:  Sampdoria
team2:  Spal
Stats before game: [108.30136083720596, 110.55125307567705], [100.77267175107326, 138.9610416125817]
	 diff:  14.959210563014466
Correct pred:  28
true result:  1
pred result:  1
form dic 1: [-1, 1, 1, 0, -1, 0, 1]
form dic 2: [1, 1, -1, 1, -1, -1, -1]
Stats AFTER game: [113.25742883134657, 92.31742346752605], [95.37190433445117, 100.96081836189707]

Result game 69:   ['Torino', 'Frosinone', 1]
team1:  Torino
team2:  Frosinone
Stats before game: [133.37797886903047, 223.04211804780476], [44.44104468618143, 168.63311826089796]
	 diff:  100.05103449112843
Correct pred:  29
true result:  1
pred result:  1
form dic 1: [-1, 0, 1, 0, -1, 0, 1, 1]
form dic 2: [-1, 0, -1, -1, -1, -1, -1, -1]
Stats AFTER game: [129.91432179246218, 208.23107883989528], [47.771010427917375, 164.14502727134393]

Result game 70:   ['Cagliari'

Stats AFTER game: [98.60305378937818, 96.39009390455391], [88.3407403216826, 75.99952969166549]

Result game 90:   ['Empoli', 'Juventus', -1]
team1:  Empoli
team2:  Juventus
Stats before game: [78.59187565647514, 271.1164401000346], [180.9163444538998, 797.6865540097398]
	 diff:  -112.39353070809388
Correct pred:  40
true result:  -1
pred result:  -1
form dic 1: [1, -1, 0, -1, -1, 0, -1, -1, 0, -1]
form dic 2: [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
Stats AFTER game: [69.53423908139828, 219.5897165737197], [207.57529177876356, 263.2250617552521]

Result game 91:   ['Torino', 'Fiorentina', 0]
team1:  Torino
team2:  Fiorentina
Stats before game: [129.91432179246218, 208.23107883989528], [95.71381316778144, 65.38073921132295]
Game ignored due to tie
Stats after game:  [129.91432179246218, 208.23107883989528], [95.71381316778144, 65.38073921132295]
Draws:  20

Result game 92:   ['Sassuolo', 'Bologna', 0]
team1:  Sassuolo
team2:  Bologna
Stats before game: [116.5657380168478, 93.32531454956457], [75

...

Stats AFTER game: [99.08790728046861, 9.99130232186123], [103.58122099406737, 11.065535970172618]

Result game 302:   ['Fiorentina', 'Frosinone', -1]
team1:  Fiorentina
team2:  Frosinone
Stats before game: [95.9421189125597, 26.560482470231367], [101.56957995980284, 18.55119659344687]
	 diff:  17.56471851209423
true result:  -1
pred result:  1
form dic 1: [1, 1, -1, 0, 1, -1, 1, -1, 0, 0, 0, 0, 0, -1, 0, 1, 1, -1, 0, 0, 1, 0, 0, 1, 0, -1, 0, -1, 0, 0, -1]
form dic 2: [-1, 0, -1, -1, -1, -1, -1, -1, 0, 1, 0, 0, -1, 0, -1, -1, 0, 0, -1, -1, 1, -1, 1, -1, -1, 0, -1, -1, -1, 1, 1]
Stats AFTER game: [93.41004612582603, 25.156588464201903], [103.58837495169206, 16.956906672524124]

Result game 303:   ['Udinese', 'Empoli', 1]
team1:  Udinese
team2:  Empoli
Stats before game: [110.70757236477621, 22.62154046696628], [109.02406112006851, 17.01389601757905]
	 diff:  -6.217534832850319
true result:  1
pred result:  -1
form dic 1: [0, 1, -1, 0, 1, -1, -1, -1, -1, 0, -1, -1, 1, 0, -1, -1, 0, 0, 1, 

Stats AFTER game: [103.98264339875786, 9.63978277809851], [100.46288014844646, 9.034249816544104]

Result game 320:   ['Parma', 'Milan', 0]
team1:  Parma
team2:  Milan
Stats before game: [96.25096783942499, 9.544864557813435], [104.18370713871509, 34.77242429362111]
Game ignored due to tie
Stats after game:  [96.25096783942499, 9.544864557813435], [104.18370713871509, 34.77242429362111]
Draws:  91

Result game 321:   ['Udinese', 'Sassuolo', 0]
team1:  Udinese
team2:  Sassuolo
Stats before game: [100.46288014844646, 9.034249816544104], [92.4663546290079, 39.3114494436732]
Game ignored due to tie
Stats after game:  [100.46288014844646, 9.034249816544104], [92.4663546290079, 39.3114494436732]
Draws:  92

Result game 322:   ['Lazio', 'Chievo', -1]
team1:  Lazio
team2:  Chievo
Stats before game: [103.98264339875786, 9.63978277809851], [57.60206254914432, 475.49737113562645]
	 diff:  78.33098450717824
true result:  -1
pred result:  1
form dic 1: [-1, -1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 0, 0, 0,

Stats AFTER game: [99.75787211677815, 7.431161420961011], [104.71476041178857, 9.836447682898683]

Result game 337:   ['Torino', 'Milan', 2]
team1:  Torino
team2:  Milan
Stats before game: [109.8688934719297, 16.94066571226294], [104.18370713871509, 34.77242429362111]
	 diff:  12.088260820125427
Correct pred:  155
true result:  2
pred result:  1
form dic 1: [-1, 0, 1, 0, -1, 0, 1, 1, 0, 0, 1, -1, 0, 1, 0, -1, 0, 1, 0, -1, 1, 0, 1, 0, 1, 1, 1, -1, 0, 1, 0, 0, 1, 1]
form dic 2: [-1, 1, 0, 0, 0, 1, 1, -1, 1, 1, 1, -1, 0, 1, 0, 0, -1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, -1, -1, 0, -1, 1, 0, -1]
Stats AFTER game: [112.18540736195209, 17.146474337425783], [98.71953046209975, 42.584261124756004]

Result game 338:   ['Atalanta', 'Udinese', 2]
team1:  Atalanta
team2:  Udinese
Stats before game: [120.70003444009927, 52.7856921630179], [100.46288014844646, 9.034249816544104]
	 diff:  74.72562568022133
Correct pred:  156
true result:  2
pred result:  1
form dic 1: [1, 0, -1, -1, 0, 0, -1, -1, 1, 1, 1, 1

Stats AFTER game: [86.92941306521985, 43.224598824664824], [112.35037828826336, 26.243080774111633]

Result game 353:   ['Torino', 'Sassuolo', 1]
team1:  Torino
team2:  Sassuolo
Stats before game: [112.18540736195209, 17.146474337425783], [96.48615410956705, 25.14444800511188]
	 diff:  49.220789274399394
Correct pred:  164
true result:  1
pred result:  1
form dic 1: [-1, 0, 1, 0, -1, 0, 1, 1, 0, 0, 1, -1, 0, 1, 0, -1, 0, 1, 0, -1, 1, 0, 1, 0, 1, 1, 1, -1, 0, 1, 0, 0, 1, 1, 0, 1]
form dic 2: [1, 0, 1, -1, 1, 1, -1, -1, 0, 0, 1, 0, -1, 0, 0, 1, 0, -1, -1, 0, 1, 0, -1, -1, 0, -1, 0, -1, -1, 1, 0, 0, 0, 1, 0, -1]
Stats AFTER game: [111.60921420803534, 17.128488052460334], [97.2907615695113, 23.99016648386544]

Result game 354:   ['Sampdoria', 'Empoli', -1]
team1:  Sampdoria
team2:  Empoli
Stats before game: [99.75787211677815, 7.431161420961011], [103.22580449368338, 21.173394017646125]
	 diff:  30.249789868319752
true result:  -1
pred result:  1
form dic 1: [-1, 1, 1, 0, -1, 0, 1, 1, 0, -

Stats AFTER game: [114.81330060240663, 13.360619224726229], [107.42807654065673, 5.023237428048094]

Result game 368:   ['Juventus', 'Atalanta', 0]
team1:  Juventus
team2:  Atalanta
Stats before game: [102.57511072686772, 20.568688127252855], [116.97402517193922, 50.101920850747995]
Game ignored due to tie
Stats after game:  [102.57511072686772, 20.568688127252855], [116.97402517193922, 50.101920850747995]
Draws:  105

Result game 369:   ['Lazio', 'Bologna', 0]
team1:  Lazio
team2:  Bologna
Stats before game: [106.8047915046145, 7.720352691096043], [108.7826438660749, 17.68298542352274]
Game ignored due to tie
Stats after game:  [106.8047915046145, 7.720352691096043], [108.7826438660749, 17.68298542352274]
Draws:  106

Result game 370:   ['Frosinone', 'Chievo', 0]
team1:  Frosinone
team2:  Chievo
Stats before game: [94.32078965011516, 12.348942864740213], [97.90690940335767, 30.856938835088012]
Game ignored due to tie
Stats after game:  [94.32078965011516, 12.348942864740213], [97.9069

In [23]:
form_dictionary

{}